In [1]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function krawczyk_simple(F,X)
    j = ForwardDiff.jacobian(F,X)
    c = mid.(X)
    R = inv(ForwardDiff.jacobian(F,c)) 
    M = Matrix{Float64}(I,size(R))-R*j
    K = c -R*F(c)+M*(X-c)
    return K
end

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = x1^2 - x2^4
F((x1,x2))=[f((x1,x2));g((x1,x2))]

X = [(0.6 .. 0.7),(0.7 .. 0.8)]
@show krawczyk_simple(F,X)


krawczyk_simple(F, X) = Interval{Float64}[[0.605296, 0.630111], [0.777629, 0.798351]]


2-element Vector{Interval{Float64}}:
 [0.605296, 0.630111]
 [0.777629, 0.798351]

In [2]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function newton(F,V,n)
    x = V
    for i = 1:n
        j = ForwardDiff.jacobian(F,x)
        R = inv(j)
        if norm(R*F(x))<10^(-3)
            return x
        end
        x = x-R*F(x)
    end
    return x
end

newton (generic function with 1 method)

In [3]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

orig_is_subset (generic function with 1 method)

In [31]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function krawczyk(F,V,n)
    x = newton(F,V,n)
    j = ForwardDiff.jacobian(F,x)
    R = inv(j)
    r = abs.(R*F(x))
    u = r.+(sum(r)/length(r))
    V2 =  x+[(-u[i] .. u[i]) for i in 1:size(R)[1]]
    
    j = ForwardDiff.jacobian(F,V2)
    c = mid.(V2)
    R = inv(ForwardDiff.jacobian(F,c)) 
    M = Matrix{Float64}(I,size(R))-R*j
    K = c -R*F(c)+M*(V2-c)
    @show V2
    @show K
    if orig_is_subset(K,V2)
        return K
    else
        return false
    end
end

# f((x1,x2)) = x1 + 2*x2 +1
# g((x1,x2)) = x1^2 + 2*(x2^2) -1

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = x1^2 - x2^4
F((x1,x2))=[f((x1,x2));g((x1,x2))]

X = [0.6, 0.8]

@show krawczyk(F,X,5)


x = [0.6181286549707602, 0.7864035087719299]
R = [0.4472733465548631 0.36161974710644257; 0.2842402474002675 -0.2842402474002675]
r = [9.471581995952683e-5, 0.0002520459125309304]
u = [0.00026809668620475546, 0.000425426778776159]
V2 = Interval{Float64}[[0.61786, 0.618397], [0.785978, 0.786829]]
K = Interval{Float64}[[0.618033, 0.618035], [0.78615, 0.786153]]
krawczyk(F, X, 5) = Interval{Float64}[[0.618033, 0.618035], [0.78615, 0.786153]]


2-element Vector{Interval{Float64}}:
 [0.618033, 0.618035]
 [0.78615, 0.786153]